In [1]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm, trange
from math import sqrt, inf
import pandas as pd
import numpy as np
import heapq
import random

In [2]:
n,m=943,1682
cols = ['user_id', 'item_id', 'rating', 'ts']
base = pd.read_csv('../../ml-100k/u1.base',sep='\t',names=cols)
test = pd.read_csv('../../ml-100k/u1.test',sep='\t',names=cols)

In [3]:
# 创建uesr-item矩阵
train_matrix = np.zeros((n, m))
test_matrix = np.zeros((n, m))
K = 50
y_ui = np.zeros((n, m))
for i in base.itertuples():
    if i[3]>=4:
        train_matrix[i[1] - 1, i[2] - 1] = i[3]

for i in test.itertuples():
    if i[3]>=4:
        test_matrix[i[1] - 1, i[2] - 1] = i[3]

# 创建y矩阵
for i in range(n):
    for j in range(m):
        if train_matrix[i,j] >= 4:
            y_ui[i,j] = 1

In [16]:
def BPR(Rui, Uuk, Vik, bik, miu, T, alpha_u, alpha_v, beta_v, gama):
    Rk = list()
    Rj = list()
    for u in range(n):
        ri = list(np.where(Rui[u,:]!=0)[0])
        rj = list(np.where(Rui[u,:]==0)[0])
        for j in ri:
            Rk.append([u,j])
        Rj.append([j for j in ri])
    Rk = np.array(Rk)# ???
    for _ in tqdm(range(T)):
        Kk = np.random.permutation(Rk)
        for k in Kk:
            ui, uj = k
            # randomly pick up an item j from J|Ju
            idx = np.random.randint(len(Rj[ui]))
            ij = Rj[ui][idx]
            # Calculate relevant parameters
            rui = np.dot(Uuk[ui,:],Vik[uj,:].T)+bik[uj]
            ruj = np.dot(Uuk[ui,:],Vik[ij,:].T)+bik[ij]
            sgm = 1/(1+np.power(np.e,(rui-ruj)))
            # Calculate the gradient of each parameters
            grad_Uu = -sgm*(Vik[uj,:]-Vik[ij,:])+alpha_u*Uuk[u,:]
            grad_Vi = -sgm*Uuk[u,:]+alpha_v*Vik[uj,:]
            grad_Vj = -sgm*Uuk[u,:]*(-1)+alpha_v*Vik[ij,:]
            grad_bi = -sgm + beta_v*bik[uj]
            grad_bj = -sgm*(-1) + beta_v*bik[ij] 
            # update the model parameters
            Uuk[u,:] -= gama*grad_Uu
            Vik[uj,:] -= gama*grad_Vi
            Vik[ij,:] -= gama*grad_Vj
            bik[uj] -= gama*grad_bi
            bik[ij] -= gama*grad_bj
    return Uuk, Vik, bik

In [17]:
# evaluate function
def PreK(rank, truth, k):
    prek = 0
    ure = 0
    for i in range(n):
        ls = list(np.where(truth[i]!=0)[0])
        ts = len(ls)
        if ts==0:
            continue
        preuk = 0
        ure+=1
        for j in range(k):
            if rank[i][j] in ls:
                preuk+=1
        prek+=preuk/k
    return prek/ure
def RecK(rank, truth, k):
    reck = 0
    ure = 0
    for i in range(n):
        ls = list(np.where(truth[i]!=0)[0])
        ts = len(ls)
        if ts==0:
            continue
        recuk = 0
        ure += 1
        for j in range(k):
            if rank[i][j] in ls:
                recuk += 1
        reck+=recuk/ts
    return reck/ure

In [18]:
if __name__ == '__main__':
    gama = 0.01
    alpha_u = alpha_v = beta_v = 0.01
    T = 500
    d = 20
    miu = y_ui.sum()/(n*m)
    b = sum(y_ui)/n - miu
    U = (np.random.rand(n, d)-0.5)*0.01
    V = (np.random.rand(m, d)-0.5)*0.01
    R = train_matrix.copy()
    U, V, b=BPR(R, U, V, b, miu, T,alpha_u, alpha_v, beta_v, gama)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [18:19<00:00,  2.20s/it]


In [19]:
rank_bpr = list()
for i in tqdm(range(n)):
    exclude = list(np.where(train_matrix[i,:]!=0)[0])
    ls = list()
    for j in range(m):
        if j in exclude:
            continue
        ls.append((j,Ruj[i,j]))
    ls = sorted(ls, key=lambda x:x[1], reverse=True)
    rank_bpr.append([ls[j][0] for j in range(len(ls))])

100%|████████████████████████████████████████████████████████████████████████████████| 943/943 [00:11<00:00, 84.05it/s]


In [20]:
print("BPR Pre@5:"+str(PreK(rank_bpr, test_matrix, 5)))
print("BPR Rec@5:"+str(RecK(rank_bpr, test_matrix, 5)))

BPR Pre@5:0.0627192982456139
BPR Rec@5:0.016011161607878657
